 # Tag:
 "Quantum Information, Science & Technology"




# Problem setup:
In quantum error correction, you encode quantum states into logical states made of many qubits in order to improve their resilience to errors. In quantum error detection, you do the same but can only detect the presence of errors and not correct them. In this problem, we will consider a single [[4,2,2]] quantum error detection code, which encodes two logical qubits into four physical qubits, and investigate how robust logical quantum operations in this code are to quantum errors.

Our convention is that the four physical qubits in the [[4,2,2]] code are labelled 0,1,2,3. The two logical qubits are labelled A and B. The stabilizers are $XXXX$ and $ZZZZ$, where $X$ and $Z$ are Pauli matrices. The logical $X$ and $Z$ operators on the two qubits are $X_A = XIXI$, $X_B=XXII$, $Z_A = ZZII$, $Z_B = ZIZI$, up to multiplication by stabilizers.

We will consider different state preparation circuits consisting of controlled not $CNOT_{ij}$ gates, where $CNOT_{ij}$ has control qubit $i$ and target qubit $j$. As a simple model of quantum errors in hardware, we will suppose that each $CNOT_{ij}$ gate in the circuit has a two qubit depolarizing error channel following it that produces one of the 15 non-identity two-qubit Paulis with equal probability $p/15$. The probability $p$ indicates the probability of an error in a single two-qubit gate. We will assess the logical infidelity of certain state preparation protocols as a function of the physical infidelity $p$.

# Main problem:

Suppose that we prepare a logical two-qubit $|00\rangle_{AB}$ state in the [[4,2,2]] code. To do so, we introduce an ancilla qubit, qubit 4, and use the following state preparation circuit:

$$M_4 (CNOT_{04}) (CNOT_{34}) (CNOT_{23}) (CNOT_{10}) (CNOT_{12}) (H_1) $$

Note that this equation is written in matrix multiplication order, while the quantum operations in the circuit occur in the reverse order (from right-to-left in the above equation).  $H$ is a single-qubit Hadamard gate and $M$ is a single-qubit measurement. The ancilla is used to detect errors in the state preparation circuit and makes the circuit fault-tolerant. If the ancilla measurement is $|0\rangle$ ($|1\rangle$), the state preparation succeeds (fails).

What is the logical state fidelity of the final 2-qubit logical state at the end of the circuit as a function of two-qubit gate error rate $p$, assuming the state is post-selected on all detectable errors in the code and on the ancilla qubit measuring $|0\rangle$?


### Parsing template:

In [ ]:
import sympy as sp

p = sp.symbols('p')

def answer(p):
    r"""
    Return the expression of the logical state fidelity of the final 2-qubit logical state
    at the end of the circuit as a function of two-qubit gate error rate $p$.

    Inputs
    ----------
    p: sympy.Symbol, two-qubit gate error rate, $p$

    Outputs
    ----------
    F_logical: sympy.Expr, logical state fidelity of the final 2-qubit logical state
    """

    # ------------------ FILL IN YOUR RESULTS BELOW ------------------
    F_logical = ...  # a SymPy expression of inputs
    # ---------------------------------------------------------------

    return F_logical

### Answer:
$
F_{\rm{logical}}=1-\frac{\frac{16}{25} p^2 - \frac{128}{125} p^3 + \frac{2048}{3375} p^4 - \frac{32768}{253125} p^5}{1 - \frac{68}{15} p + \frac{704}{75} p^2 - \frac{32768}{3375} p^3 + \frac{
 253952}{50625} p^4 - \frac{262144}{253125} p^5}
$

### Answer code:

In [ ]:
F_logical = 1 - (sp.Rational(16, 25)* p**2 - sp.Rational(128, 125)* p**3 + sp.Rational(2048, 3375)* p**4 - sp.Rational(32768, 253125)* p**5) / (1 - sp.Rational(68, 15)* p + sp.Rational(704, 75)* p**2 - sp.Rational(32768, 3375)* p**3 + sp.Rational(253952, 50625)* p**4 - sp.Rational(262144, 253125)* p**5)